## Dependencies

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import multiprocessing as mp

In [2]:
train = pd.read_csv('../output/data_5_fold.csv')
test = pd.read_csv('../output/test.csv')

print('Number of train samples: ', train.shape[0])
print('Number of test samples: ', test.shape[0])
display(test.head())

Number of train samples:  19681
Number of test samples:  3140


,Unnamed: 0,id_code,diagnosis
0,0,10_left.jpeg,0
1,1,10_right.jpeg,0
2,2,13_left.jpeg,0
3,3,13_right.jpeg,0
4,4,17_left.jpeg,0


# Define variable

In [3]:
HEIGHT = 224
WIDTH = 224


data_2_path = '../data/dr_data_2/train/'
data_1_path = '../data/dr_data_1/train/'
test_path = '../data/dr_test/test/'
train_dest_path = '../dataset/train/'
test_dest_path =  '../dataset/test/'

# Pre-procecess images

In [4]:
def crop_image(img, tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
            
        return img

def circle_crop(img):
    img = crop_image(img)

    height, width, depth = img.shape
    largest_side = np.max((height, width))
    img = cv2.resize(img, (largest_side, largest_side))

    height, width, depth = img.shape

    x = width//2
    y = height//2
    r = np.amin((x, y))

    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x, y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image(img)

    return img
        
def preprocess_image(image_id, base_path, save_path, HEIGHT=HEIGHT, WIDTH=WIDTH, sigmaX=10):
    print(base_path + image_id)
    image = cv2.imread(base_path + image_id)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = circle_crop(image)
    image = cv2.resize(image, (HEIGHT, WIDTH))
    cv2.imwrite(save_path + image_id, image)
    

def preprocess_data(df, HEIGHT=HEIGHT, WIDTH=WIDTH, sigmaX=10):
    df = df.reset_index()
    print("Train images")
    for i in range(df.shape[0]):
        item = df.iloc[i]
        image_id = item['id_code']
        item_data = item['data']
        if item_data == 1:
            preprocess_image(image_id, data_1_path, train_dest_path)
        if item_data == 2:
            preprocess_image(image_id, data_2_path, train_dest_path)

        
def preprocess_test(df, base_path=test_path, save_path=test_dest_path, HEIGHT=HEIGHT, WIDTH=WIDTH, sigmaX=10):
    df = df.reset_index()
    print("Test images")
    for i in range(df.shape[0]):
        image_id = df.iloc[i]['id_code']
        preprocess_image(image_id, base_path, save_path)


In [5]:
preprocess_test(test)
preprocess_data(train)

Test images
../data/dr_test/test/10_left.jpeg
../data/dr_test/test/10_right.jpeg
../data/dr_test/test/13_left.jpeg
../data/dr_test/test/13_right.jpeg
../data/dr_test/test/17_left.jpeg
../data/dr_test/test/19_left.jpeg
../data/dr_test/test/19_right.jpeg
../data/dr_test/test/20_left.jpeg
../data/dr_test/test/20_right.jpeg
../data/dr_test/test/21_left.jpeg
../data/dr_test/test/21_right.jpeg
../data/dr_test/test/22_left.jpeg
../data/dr_test/test/22_right.jpeg
../data/dr_test/test/23_left.jpeg
../data/dr_test/test/23_right.jpeg
../data/dr_test/test/25_left.jpeg
../data/dr_test/test/25_right.jpeg
../data/dr_test/test/31_left.jpeg
../data/dr_test/test/31_right.jpeg
../data/dr_test/test/33_left.jpeg
../data/dr_test/test/33_right.jpeg
../data/dr_test/test/36_right.jpeg
../data/dr_test/test/40_right.jpeg
../data/dr_test/test/41_left.jpeg
../data/dr_test/test/41_right.jpeg
../data/dr_test/test/42_left.jpeg
../data/dr_test/test/42_right.jpeg
../data/dr_test/test/46_left.jpeg
../data/dr_test/test/4

../data/dr_test/test/376_left.jpeg
../data/dr_test/test/376_right.jpeg
../data/dr_test/test/380_left.jpeg
../data/dr_test/test/380_right.jpeg
../data/dr_test/test/381_left.jpeg
../data/dr_test/test/381_right.jpeg
../data/dr_test/test/384_left.jpeg
../data/dr_test/test/384_right.jpeg
../data/dr_test/test/386_left.jpeg
../data/dr_test/test/386_right.jpeg
../data/dr_test/test/389_left.jpeg
../data/dr_test/test/389_right.jpeg
../data/dr_test/test/393_left.jpeg
../data/dr_test/test/395_left.jpeg
../data/dr_test/test/395_right.jpeg
../data/dr_test/test/398_left.jpeg
../data/dr_test/test/398_right.jpeg
../data/dr_test/test/399_left.jpeg
../data/dr_test/test/399_right.jpeg
../data/dr_test/test/400_left.jpeg
../data/dr_test/test/400_right.jpeg
../data/dr_test/test/404_left.jpeg
../data/dr_test/test/404_right.jpeg
../data/dr_test/test/408_left.jpeg
../data/dr_test/test/408_right.jpeg
../data/dr_test/test/410_left.jpeg
../data/dr_test/test/410_right.jpeg
../data/dr_test/test/412_left.jpeg
../data

../data/dr_test/test/773_left.jpeg
../data/dr_test/test/773_right.jpeg
../data/dr_test/test/780_left.jpeg
../data/dr_test/test/780_right.jpeg
../data/dr_test/test/783_left.jpeg
../data/dr_test/test/783_right.jpeg
../data/dr_test/test/784_left.jpeg
../data/dr_test/test/784_right.jpeg
../data/dr_test/test/787_left.jpeg
../data/dr_test/test/787_right.jpeg
../data/dr_test/test/791_left.jpeg
../data/dr_test/test/791_right.jpeg
../data/dr_test/test/792_left.jpeg
../data/dr_test/test/792_right.jpeg
../data/dr_test/test/793_left.jpeg
../data/dr_test/test/793_right.jpeg
../data/dr_test/test/796_left.jpeg
../data/dr_test/test/796_right.jpeg
../data/dr_test/test/799_left.jpeg
../data/dr_test/test/799_right.jpeg
../data/dr_test/test/800_left.jpeg
../data/dr_test/test/800_right.jpeg
../data/dr_test/test/802_left.jpeg
../data/dr_test/test/802_right.jpeg
../data/dr_test/test/803_left.jpeg
../data/dr_test/test/803_right.jpeg
../data/dr_test/test/805_left.jpeg
../data/dr_test/test/805_right.jpeg
../dat

../data/dr_test/test/1161_left.jpeg
../data/dr_test/test/1161_right.jpeg
../data/dr_test/test/1165_left.jpeg
../data/dr_test/test/1165_right.jpeg
../data/dr_test/test/1170_left.jpeg
../data/dr_test/test/1170_right.jpeg
../data/dr_test/test/1172_left.jpeg
../data/dr_test/test/1172_right.jpeg
../data/dr_test/test/1179_left.jpeg
../data/dr_test/test/1179_right.jpeg
../data/dr_test/test/1180_left.jpeg
../data/dr_test/test/1180_right.jpeg
../data/dr_test/test/1182_left.jpeg
../data/dr_test/test/1182_right.jpeg
../data/dr_test/test/1188_left.jpeg
../data/dr_test/test/1188_right.jpeg
../data/dr_test/test/1192_left.jpeg
../data/dr_test/test/1192_right.jpeg
../data/dr_test/test/1197_left.jpeg
../data/dr_test/test/1197_right.jpeg
../data/dr_test/test/1199_left.jpeg
../data/dr_test/test/1199_right.jpeg
../data/dr_test/test/1202_left.jpeg
../data/dr_test/test/1202_right.jpeg
../data/dr_test/test/1206_left.jpeg
../data/dr_test/test/1206_right.jpeg
../data/dr_test/test/1211_left.jpeg
../data/dr_test

../data/dr_test/test/1570_right.jpeg
../data/dr_test/test/1574_left.jpeg
../data/dr_test/test/1574_right.jpeg
../data/dr_test/test/1579_left.jpeg
../data/dr_test/test/1579_right.jpeg
../data/dr_test/test/1586_left.jpeg
../data/dr_test/test/1586_right.jpeg
../data/dr_test/test/1588_left.jpeg
../data/dr_test/test/1588_right.jpeg
../data/dr_test/test/1589_left.jpeg
../data/dr_test/test/1589_right.jpeg
../data/dr_test/test/1595_left.jpeg
../data/dr_test/test/1595_right.jpeg
../data/dr_test/test/1596_left.jpeg
../data/dr_test/test/1596_right.jpeg
../data/dr_test/test/1600_left.jpeg
../data/dr_test/test/1600_right.jpeg
../data/dr_test/test/1608_left.jpeg
../data/dr_test/test/1608_right.jpeg
../data/dr_test/test/1609_left.jpeg
../data/dr_test/test/1609_right.jpeg
../data/dr_test/test/1610_left.jpeg
../data/dr_test/test/1610_right.jpeg
../data/dr_test/test/1611_left.jpeg
../data/dr_test/test/1611_right.jpeg
../data/dr_test/test/1624_left.jpeg
../data/dr_test/test/1624_right.jpeg
../data/dr_tes

...

../data/dr_data_1/train/b8e9a8f4617d.png
../data/dr_data_1/train/b8ebedd382de.png
../data/dr_data_1/train/b8f1b30877db.png
../data/dr_data_1/train/b8fb9f55cd6d.png
../data/dr_data_1/train/b90bc89ce8d8.png
../data/dr_data_1/train/b9127e38d9b9.png
../data/dr_data_1/train/b91ef82e723a.png
../data/dr_data_1/train/b927a9238434.png
../data/dr_data_1/train/b92eacd1392a.png
../data/dr_data_1/train/b94c58d063bf.png
../data/dr_data_1/train/b9519abce0c1.png
../data/dr_data_1/train/b95d4dd8e5e2.png
../data/dr_data_1/train/b960142a8de7.png
../data/dr_data_1/train/b963a11638f2.png
../data/dr_data_1/train/b96b518596b3.png
../data/dr_data_1/train/b98f77098b9d.png
../data/dr_data_1/train/b99794a0beed.png
../data/dr_data_1/train/b99afe7137fb.png
../data/dr_data_1/train/b99c825b93c5.png
../data/dr_data_1/train/b9b6ee2b9453.png
../data/dr_data_1/train/b9b99dad668d.png
../data/dr_data_1/train/b9bc81fcb075.png
../data/dr_data_1/train/b9c7c5182075.png
../data/dr_data_1/train/b9d0b83d70c3.png
../data/dr_data_

../data/dr_data_1/train/c6a8f8f998a2.png
../data/dr_data_1/train/c6c2bad91f23.png
../data/dr_data_1/train/c6d4e4a3bd4c.png
../data/dr_data_1/train/c6e1e9fbf39b.png
../data/dr_data_1/train/c6f5b5b5be41.png
../data/dr_data_1/train/c704bd669f36.png
../data/dr_data_1/train/c70d09370109.png
../data/dr_data_1/train/c739ff9580d3.png
../data/dr_data_1/train/c73c5f6ef664.png
../data/dr_data_1/train/c755a0c4edcc.png
../data/dr_data_1/train/c76664770c07.png
../data/dr_data_1/train/c7b622ec8104.png
../data/dr_data_1/train/c7c0470bcf87.png
../data/dr_data_1/train/c7c3d363bc86.png
../data/dr_data_1/train/c7d0deb71576.png
../data/dr_data_1/train/c7e827fc7f41.png
../data/dr_data_1/train/c80b0f27541a.png
../data/dr_data_1/train/c80f79579fed.png
../data/dr_data_1/train/c81c6911f5e0.png
../data/dr_data_1/train/c85b79d70079.png
../data/dr_data_1/train/c87493ed320c.png
../data/dr_data_1/train/c8823cdaf7fa.png
../data/dr_data_1/train/c8905b8d5cf1.png
../data/dr_data_1/train/c8a3eb9a5b52.png
../data/dr_data_

../data/dr_data_1/train/d56d32a1d62d.png
../data/dr_data_1/train/d57d1be1bbd1.png
../data/dr_data_1/train/d5a39339ff3d.png
../data/dr_data_1/train/d5ad3362424c.png
../data/dr_data_1/train/d5b4705ac2ee.png
../data/dr_data_1/train/d5c63a8d9e94.png
../data/dr_data_1/train/d6130f2ec903.png
../data/dr_data_1/train/d6228d951958.png
../data/dr_data_1/train/d6283ded6aea.png
../data/dr_data_1/train/d659d7fd5ccf.png
../data/dr_data_1/train/d667af5742f6.png
../data/dr_data_1/train/d66b6f333dc7.png
../data/dr_data_1/train/d66ccb75ada1.png
../data/dr_data_1/train/d67374d3fa2a.png
../data/dr_data_1/train/d6803e467592.png
../data/dr_data_1/train/d69698f838db.png
../data/dr_data_1/train/d6b109c82067.png
../data/dr_data_1/train/d6dbb0820ea5.png
../data/dr_data_1/train/d6df4fe492ec.png
../data/dr_data_1/train/d6e26fe51dce.png
../data/dr_data_1/train/d6f36ec5564a.png
../data/dr_data_1/train/d6f6bdfd8011.png
../data/dr_data_1/train/d7078e8b0349.png
../data/dr_data_1/train/d74ccc796517.png
../data/dr_data_

../data/dr_data_1/train/e30a890600e1.png
../data/dr_data_1/train/e31c42a8652b.png
../data/dr_data_1/train/e322acd46152.png
../data/dr_data_1/train/e32a359be36d.png
../data/dr_data_1/train/e32dc722eca5.png
../data/dr_data_1/train/e33766353db2.png
../data/dr_data_1/train/e34fa07bd64d.png
../data/dr_data_1/train/e387311a840e.png
../data/dr_data_1/train/e38f3a65b02b.png
../data/dr_data_1/train/e39b627cf648.png
../data/dr_data_1/train/e3a7671f787b.png
../data/dr_data_1/train/e3ab63dc9a60.png
../data/dr_data_1/train/e3b47ed5b511.png
../data/dr_data_1/train/e3cd96cb094c.png
../data/dr_data_1/train/e3e490babc0c.png
../data/dr_data_1/train/e3ec668f6fad.png
../data/dr_data_1/train/e4151feb8443.png
../data/dr_data_1/train/e4210e7fe587.png
../data/dr_data_1/train/e42d9a94a66d.png
../data/dr_data_1/train/e4730ddde408.png
../data/dr_data_1/train/e47452069ea1.png
../data/dr_data_1/train/e47770a2e5d1.png
../data/dr_data_1/train/e499434242cc.png
../data/dr_data_1/train/e4a44f9158dc.png
../data/dr_data_

../data/dr_data_1/train/f09fd9433dff.png
../data/dr_data_1/train/f0a2dc580009.png
../data/dr_data_1/train/f0c0f7b5e820.png
../data/dr_data_1/train/f0c13be90519.png
../data/dr_data_1/train/f0e1201b5c1f.png
../data/dr_data_1/train/f0f89314e860.png
../data/dr_data_1/train/f18abfa690ab.png
../data/dr_data_1/train/f1979147aad4.png
../data/dr_data_1/train/f1a761c68559.png
../data/dr_data_1/train/f1d719c97838.png
../data/dr_data_1/train/f1dc26c4bfa3.png
../data/dr_data_1/train/f2094a20b275.png
../data/dr_data_1/train/f233638e0e90.png
../data/dr_data_1/train/f23902998c21.png
../data/dr_data_1/train/f252046c0fe6.png
../data/dr_data_1/train/f26b02ead915.png
../data/dr_data_1/train/f298b7d05958.png
../data/dr_data_1/train/f2c0b41acd05.png
../data/dr_data_1/train/f2d2a0c92034.png
../data/dr_data_1/train/f2ee81781411.png
../data/dr_data_1/train/f2f569a64949.png
../data/dr_data_1/train/f30f203ef51e.png
../data/dr_data_1/train/f35d80bb1a22.png
../data/dr_data_1/train/f361060eda3e.png
../data/dr_data_